<a href="https://colab.research.google.com/github/YuLiu83/Practical-business-case-code-demo/blob/main/Merchandise_product_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Background:

A recommendation engine is a system that suggests products, services, information to users based on analysis of data. Recommendation systems are quickly becoming the primary way for users to expose to the whole digital world through the lens of their experiences, behaviours, preferences and interests. And in a world of information density and product overload, a recommendation engine provides an efficient way for companies to provide consumers with personalised information and solutions.

In the last decade, industry has developed and implemented many well known personalized recommendation algorithms, such as:

*   Traditional approaches: User/Item based CF, Content-based, FM, GB-LR
*   Deep-learning: Deep&Wide, Deep&Cross, DeepFM

The purpose of this case study is to build a simple recommendation engine for an UK online merchandise based on its transactional data. A powerful graph data architecture and a random-walk driven algorithm (personalized pagerank algorithm-PPR) is selected to achieve this goal.

(PPR algorithm computes the relevance of the nodes in a graph (In our case, 'Invoice' and 'Product' are nodes in the graph and 'Price' is the weight on the edge-- i.e. more expensive products get higher chance to be recommended). The recommendation process uses these relevance scores to estimate the likelihood of an unseen item being accessed by one specific user)

### Load package and transactional data:

In [ ]:
import pandas as pd 
import networkx as nx
import matplotlib.pyplot as plt
import operator


In [ ]:
# Load google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data=pd.read_excel('/content/drive/My Drive/Product recommendation/Online Retail.xlsx')

In [ ]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


### Data processing:

In [ ]:
UK_data=data[data['Country']=='United Kingdom']

In [ ]:
print('# of unique trasactions: {}'.format(UK_data.InvoiceNo.nunique()))
print('# of unique purchased products: {}'.format(UK_data.StockCode.nunique()))
print('Date range from {} to {} for {}'.format(UK_data.InvoiceDate.min().date(), UK_data.InvoiceDate.max().date(), (UK_data.InvoiceDate.max().date()-UK_data.InvoiceDate.min().date())))

# of unique trasactions: 23494
# of unique purchased products: 4065
Date range from 2010-12-01 to 2011-12-09 for 373 days, 0:00:00


In [ ]:
print(UK_data['UnitPrice'].describe())
print('99% qunatile: {}'.format(UK_data['UnitPrice'].quantile(0.99)))

count    495478.000000
mean          4.532422
std          99.315438
min      -11062.060000
25%           1.250000
50%           2.100000
75%           4.130000
max       38970.000000
Name: UnitPrice, dtype: float64
99% qunatile: 16.95


In [ ]:
# select items priced greater than $0 and less than $17
# take off missing rows

UK_data=data[(data['UnitPrice']>0.1) & (data['UnitPrice']<17)]
UK_data=UK_data.dropna()
UK_data.reset_index(drop=True, inplace=True)

In [ ]:
UK_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Adjusted_Price
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2.891367
48,536373,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:02:00,2.55,17850.0,United Kingdom,2.891367
65,536375,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:32:00,2.55,17850.0,United Kingdom,2.891367
216,536390,85123A,WHITE HANGING HEART T-LIGHT HOLDER,64,2010-12-01 10:19:00,2.55,17511.0,United Kingdom,2.891367
257,536394,85123A,WHITE HANGING HEART T-LIGHT HOLDER,32,2010-12-01 10:39:00,2.55,13408.0,United Kingdom,2.891367


In [ ]:
print(UK_data['UnitPrice'].describe())
print('99% qunatile: {}'.format(UK_data['UnitPrice'].quantile(0.99)))

count    404259.000000
mean          2.801180
std           2.719944
min           0.110000
25%           1.250000
50%           1.950000
75%           3.750000
max          16.950000
Name: UnitPrice, dtype: float64
99% qunatile: 12.75


In [ ]:
print('Number of Unique Customers: {}'.format(UK_data['InvoiceNo'].nunique()))
print('Number of Unique Products: {}'.format(UK_data['StockCode'].nunique()))

Number of Unique Customers: 21840
Number of Unique Products: 3653


In [ ]:
# Price may change for same item, take the average purchase prices for each product

Adjusted_Price=UK_data['UnitPrice'].groupby(UK_data['StockCode']).mean().rename('Adjusted_Price')

In [ ]:
UK_data=pd.merge(UK_data, Adjusted_Price, how='inner', on=None, left_on='StockCode', right_on='StockCode',
         left_index=False, right_index=True, sort=False,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

In [ ]:
UK_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 404259 entries, 0 to 403785
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   InvoiceNo       404259 non-null  object        
 1   StockCode       404259 non-null  object        
 2   Description     404259 non-null  object        
 3   Quantity        404259 non-null  int64         
 4   InvoiceDate     404259 non-null  datetime64[ns]
 5   UnitPrice       404259 non-null  float64       
 6   CustomerID      404259 non-null  float64       
 7   Country         404259 non-null  object        
 8   Adjusted_Price  404259 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 30.8+ MB


In [ ]:
# Convert data type to string

convert_dict = {'InvoiceNo': pd.StringDtype(), 
                'StockCode': pd.StringDtype()} 
UK_data = UK_data.astype(convert_dict) 


In [ ]:
print(UK_data.dtypes)

InvoiceNo                 string
StockCode                 string
Description               object
Quantity                   int64
InvoiceDate       datetime64[ns]
UnitPrice                float64
CustomerID               float64
Country                   object
Adjusted_Price           float64
dtype: object


In [ ]:
# This example shows same StockCode was charged for different prices during the year
UK_data['UnitPrice'][UK_data['StockCode']=='85123A'].unique()

array([2.55, 2.95, 3.24, 2.4 , 3.2 , 5.79])

Convert data into graph format where 'InvoiceNo' and 'StockCode' are graph nodes and 'Adjusted_Price' function as weights over the edge between nodes:

In [ ]:
import networkx as nx
G = nx.Graph()
G = nx.from_pandas_edgelist(UK_data, 'StockCode', 'InvoiceNo', ['Adjusted_Price'])


### Build recommendation function:

In [ ]:
def Recommend(invoice):

  ppr = nx.pagerank(G, personalization={str(invoice): 1}, weight='Adjusted_Price', max_iter=100, tol=1e-06, nstart=None, dangling=None) # for PPR, 100% back to the starting point
  ppr = sorted(ppr.items(), key=operator.itemgetter(1), reverse=True)

  Purchased=list(UK_data['StockCode'][UK_data['InvoiceNo']==str(invoice)])

  Invoice=list(UK_data['InvoiceNo'])
  Item_list=[]
  n=1
  for item, score in ppr:
      if item not in Invoice and item not in Purchased:
        n+=1
        Item_list.append(str(item))
        #print(item, score)
        if n>5:
          return pd.DataFrame({'Top-5 Recommended Products': UK_data[UK_data['StockCode'].isin(Item_list)]['Description'].unique()})
          

### Product recommnedation test cases:

Example 1: Top 5 products recommended based on products purchased with invoice number '536365'

In [ ]:
Recommend(536365)

,Top-5 Recommended Products
0,WOOD 2 DRAWER CABINET WHITE FINISH
1,WOOD S/3 CABINET ANT WHITE FINISH
2,3 DRAWER ANTIQUE WHITE WOOD CABINET
3,REGENCY CAKESTAND 3 TIER
4,CREAM SWEETHEART MINI CHEST
5,PARTY BUNTING


The recommendation program finds over-sell opportunities on kitchen furnitures to the customer who made this transaction. 

There are six items recommended by the algorithm instead of five. A closer look suggests the 'WOOD S/3 CABINET ANT WHITE FINISH' and '3 DRAWER ANTIQUE WHITE WOOD CABINET' may be the same product. Further dat cleaning will required to unify item names, making one to one relation between StockCode and Description.

Example 2: Top 5 products recommended based on products purchased with invoice number '542992'

In [ ]:
Recommend(542992)

,Top-5 Recommended Products
0,JAM MAKING SET WITH JARS
1,REGENCY CAKESTAND 3 TIER
2,RED RETROSPOT CAKE STAND
3,ROMANTIC IMAGES NOTEBOOK SET
4,BLOSSOM IMAGES NOTEBOOK SET


In this example, products recommended relating to this purchase are for cake making and memmory keeping.

### Discussion:

Previous examples show our random walk based recommender algorithm produces meaningful product recommendations. However, due to data limitation, there are a few drawbacks for our recommender engine:

*   The algorithm can only recommend items being purchased, products were not shown in the transaction data cannot be recommended.
*   Since the total number of different products at store is unknown, off-line metrics such as 'diversity' cannot be calculated.

In a deeper analysis, off-line evaluation metrics such as MAP maybe used to test the accuracy of our algorithm, in which case a train-test split and cross-validation step maybe needed.
